In [1]:
import pandas as pd
import numpy as np
from scipy import stats
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.utils import shuffle
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from xgboost import XGBRegressor
import os
import pickle
from datetime import datetime
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import time

import warnings

# התעלמות מכל ההתראות
warnings.filterwarnings("ignore")


#pd.set_option('display.max_rows', None)


# יצירה של פייפליין

### קריאה קובץ לבדיקה פנימית

In [ ]:
path ='InputData\Panel_06.csv'
panel_05_df = pd.read_csv(path)
#panel_05_df.isnull().sum()


## פונקציה מספר 1 - בחירת משתנה מטרה

In [ ]:
"""def SelectTarget(df, disease):
    columns_to_drop = []
    if disease == 'AST':
        print("Selected case: AST \n")
        columns_to_drop = [col for col in df.columns if col.startswith('COPD')]
        
        # Remove rows where AST column is NaN
        df = df.dropna(subset=['AST_Prevalence'])
        
    elif disease == 'COPD':
        print("Selected case: COPD")
        columns_to_drop = [col for col in df.columns if col.startswith('AST')]
        
        # Remove rows where COPD column is NaN
        df = df.dropna(subset=['COPD_Prevalence'])
        
    else:
        print("Error: No matching case found for the disease provided.")
        return None

    #print(f"Columns to drop: {columns_to_drop}")

    updated_df = df.drop(columns=columns_to_drop)

    return updated_df"""


### בדיקה פנימית

In [ ]:
"""df_after_SelectTarget = SelectTarget(panel_05_df,'AST')
df_after_SelectTarget.head(2)"""

## פונקציה מספר 2 - החזרת דטא פריים לפי פיצ'רים נבחרים

חלוקה לקטגוריות

In [2]:
feature_categories = {
    "Gender": {
        "description": "Demographic data based on gender",
        "columns": [
            "Total_All", "MALE_All", "FEMALE_All", 
            "Male_0_4", "Male_5_9", "Male_10_14", "Male_15_19", "Male_20_24", 
            "Male_25_29", "Male_30_34", "Male_35_39", "Male_40_44", "Male_45_49", 
            "Male_50_54", "Male_55_59", "Male_60_64", "Male_65_69", "Male_70_74", 
            "Male_75_79", "Male_80_84", "Male_85+",
            "Female_0_4", "Female_5_9", "Female_10_14", "Female_15_19", "Female_20_24", 
            "Female_25_29", "Female_30_34", "Female_35_39", "Female_40_44", "Female_45_49", 
            "Female_50_54", "Female_55_59", "Female_60_64", "Female_65_69", "Female_70_74", 
            "Female_75_79", "Female_80_84", "Female_85+"
        ]
    },
    "Asthma": {
        "description": "Asthma-related data",
        "columns": [
             "AST002(percent)", "AST003(percent)", "AST004(percent)"
        ]
    },
    "COPD": {
        "description": "Chronic Obstructive Pulmonary Disease (COPD) related data",
        "columns": [
             "COPD002(percent)", "COPD003(percent)", 
            "COPD004(percent)", "COPD005(percent)", "COPD007(percent)"
        ]
    },
    "Income": {
        "description": "Economic data related to income",
        "columns": [
            "Weighted_Total_annual_income_(£)", "Weighted_Net_annual_income_(£)", 
            "Weighted_Net_annual_income_before_housing_costs_(£)", 
            "Weighted_Net_annual_income_after_housing_costs_(£)"
        ]
    },
    "Deprivation": {
        "description": "Data related to deprivation index",
        "columns": [
            "Weighted_Index_of_Multiple_Deprivation_(IMD)", "Weighted_Income", 
            "Weighted_Employment", "Weighted_Education,_Skills_and_Training", 
            "Weighted_Health_Deprivation_and_Disability", "Weighted_Crime", 
            "Weighted_Barriers_to_Housing_and_Services", "Weighted_Living_Environment"
        ]
    },
    "Parks_and_Housing": {
        "description": "Data related to parks and housing",
        "columns": [
            "Weighted_ParksOnly_Average_distance_to_nearest_Park_or_Public_Garden_(m)",
            "Weighted_ParksOnly_Average_size_of_nearest_Park_or_Public_Garden_(m2)",
            "Weighted_ParksOnly_Average_number_of_Parks_or_Public_Gardens_within_1,000_m_radius",
            "Weighted_ParksOnly_Number_of_built_up_area_postcodes__within_300m_of_a_Park_or_Public_Garden_(percentage)",
            "Weighted_ParksOnly_Number_of_built_up_area_postcodes__within_900m_of_a_Park_or_Public_Garden_(percentage)",
            "Weighted_ParksAndPlayingFields_Average_distance_to_nearest_Park_Public_Garden_or_Playing_Field_(m)",
            "Weighted_ParksAndPlayingFields_Average_size_of_nearest_Park_Public_Garden_or_Playing_Field_(m2)",
            "Weighted_ParksAndPlayingFields_Average_number_of__Parks_Public_Gardens_or_Playing_Fields_within_1,000_m_radius",
            "Weighted_ParksAndPlayingFields_Number_of_built_up_area_postcodes__within_300m_of_a_Park_Public_Garden_or_Playing_Field_(percentage)",
            "Weighted_ParksAndPlayingFields_Number_of_built_up_area_postcodes__within_900m_of_a_Park_Public_Garden_or_Playing_Field_(percentage)",
            "Weighted_Houses_Percentage_of_adresses_with_private_outdoor_space",
            "Weighted_Houses_Average_size_of_private_outdoor_space_(m2)",
            "Weighted_Houses_Median_size_of_private_outdoor_space_(m2)",
            "Weighted_Flats_Percentage_of_adresses_with_private_outdoor_space",
            "Weighted_Flats_Average_size_of_private_outdoor_space_(m2)",
            "Weighted_Flats_Average_number_of_flats_sharing_a_garden",
            "Weighted_Total_Percentage_of_adresses_with_private_outdoor_space"
        ]
    },
    "Obesity": {
        "description": "Obesity-related data",
        "columns": [
            "Obesity_Prevalence(per_cent)"
        ]
    },
    "Smoking": {
        "description": "Smoking-related data",
        "columns": [
            "SMOK002(percent)",
            "SMOK004(percent)",
            "SMOK005(percent)"
        ]
    }
}


In [3]:
def SelectFeatures(df, feature_types, disease):
    Target = ""
    if disease == 'AST':
        Target = 'AST_Prevalence'
    elif disease == 'COPD':
        Target = 'COPD_Prevalence'
    else:
        print("Error: No matching case found for the disease provided.")
        return None
    
    print("Selected Categories :\n")
    selected_features = []
    

    for feature_type in feature_types:
        if feature_type in feature_categories:
            selected_features.extend(feature_categories[feature_type]["columns"])
            print(f"{feature_type}:")
            print(", ".join(feature_categories[feature_type]["columns"]))
            print(f"Description: {feature_categories[feature_type]['description']}\n")
        else:
            print(f"Error: Feature type '{feature_type}' not found!")

    selected_features_df = df[selected_features].copy()  # Make a copy of the selected features
    selected_features_df['YEAR'] = df['YEAR']
    selected_features_df[Target] = df[Target]

    return selected_features_df

### בדיקה פנימית

In [ ]:
SelectedCategories = ['Asthma', 'Smoking']
df_after_SelectFeatures = SelectFeatures(panel_05_df ,SelectedCategories , 'AST' )
df_after_SelectFeatures.head(2)


## פונקציה מספר 3 - ניקוי חריגים

In [4]:
def OutliersRemoveOrReplace(df, method_outliers, threshold, action):
    """
    Clean outliers from a DataFrame.
    
    Parameters:
    df (pd.DataFrame): The input DataFrame.
    method_outliers (str): The method to use for outlier detection ('IQR' or 'StdDev').
    threshold (float): The threshold for determining outliers.
    action (str): The action to take on outliers ('remove' or 'replace'). 
    
    Returns:
    pd.DataFrame: The DataFrame with outliers handled.
    """
    # Select only numeric columns for outlier detection
    numeric_df = df.select_dtypes(include=[np.number])

    if method_outliers == 'IQR':
        Q1 = numeric_df.quantile(0.25)
        Q3 = numeric_df.quantile(0.75)
        IQR = Q3 - Q1
        filter_condition = ~((numeric_df < (Q1 - threshold * IQR)) | (numeric_df > (Q3 + threshold * IQR))).any(axis=1)
        outlier_condition = ((numeric_df < (Q1 - threshold * IQR)) | (numeric_df > (Q3 + threshold * IQR)))
    elif method_outliers == 'StdDev':
        mean = numeric_df.mean()
        std_dev = numeric_df.std()
        filter_condition = ~((numeric_df < (mean - threshold * std_dev)) | (numeric_df > (mean + threshold * std_dev))).any(axis=1)
        outlier_condition = ((numeric_df < (mean - threshold * std_dev)) | (numeric_df > (mean + threshold * std_dev)))
    else:
        raise ValueError("Invalid method. Choose 'IQR' or 'StdDev'.")

    if action == 'remove':
        # Filter the original dataframe to remove outliers
        df_clean = df[filter_condition]
        
        print("Clean outliers method:", method_outliers)
        print("Threshold:", threshold)
        print("Action: remove outliers")
        print(f"Number of outliers removed: {len(df) - len(df_clean)}")
        print(f"Number of rows in the cleaned dataset: {len(df_clean)}\n")
        
        return df_clean
    
    elif action == 'replace':
        # Replace outliers with NaN
        df_replaced = df.copy()
        df_replaced[outlier_condition] = np.nan
        
        print("Clean outliers method:", method_outliers)
        print("Threshold:", threshold)
        print("Action: replace outliers with NaN")
        print(f"Number of outliers replaced with NaN: {outlier_condition.sum().sum()}")
        
        return df_replaced
    
    else:
        raise ValueError("Invalid action. Choose 'remove' or 'replace'.")


### בדיקה פנימית

In [ ]:
method_outliers='IQR'
threshold = 1.5
Outliersaction='replace'
df_after_Clean_outliers = OutliersRemoveOrReplace(df_after_SelectFeatures, method_outliers, threshold, Outliersaction)
( df_after_SelectFeatures,method_outliers, threshold)
df_after_Clean_outliers.head(2)

### בדיקה של ערכים חריגים

In [ ]:
Data_chek_outliers = {
    'Apple': np.array([100, 150, 10., 40., 60., 50., 60., 60., 50., 20., 100., 70., 40., 30., 100., 100., 60., 40., 10., 50.]),
    'Banana': np.array([50., 45., 2, 30., 15., 10., 45., 15., 15., 50., 15., 10., -15, 15., 10., 50., 500, 15., 100, 32]),
    'Cherry': np.array([100., 70., 30., 80., 70., 40., 40., 1000, 100., 15, 30., 100., 30., 10., 90., 40., 50., 70., 50, 30.]),
    'Pear': np.array([45., 55., 5., 35., 35., 35., 45., 35., 55., 25., 0, 45., 75., 25., 55., 15., 95., 75., 75., 45.]),
    'Elderberry': np.array([48., 84., 96., 48., 48., 24., 102., 96., 84., 108., 12., 60., 96., 36., 36., 60., 36., 72., 96., 72.])
}

df_chek_outliers = pd.DataFrame(Data_chek_outliers)
df_chek_outliers

In [ ]:
"""הפונקציה calculate_outlier_bounds 
מקבלת דאטה פריים ומחזירה מילון שבו לכל עמודה יש גבולות עליונים ותחתונים לפי שיטת הזיהוי שנבחרה 
(IQR, Z-score או StdDev)."""

def calculate_outlier_bounds(df, method='IQR', threshold=1.5):
    outlier_bounds = {}
    
    for column in df.columns:
        data = df[column]
        
        if method == 'IQR':
            Q1 = data.quantile(0.25)
            Q3 = data.quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - threshold * IQR
            upper_bound = Q3 + threshold * IQR
        elif method == 'Z-score':
            z_scores = np.abs(stats.zscore(data))
            lower_bound = np.mean(data) - threshold * np.std(data)
            upper_bound = np.mean(data) + threshold * np.std(data)
        elif method == 'StdDev':
            mean = data.mean()
            std_dev = data.std()
            lower_bound = mean - threshold * std_dev
            upper_bound = mean + threshold * std_dev
        else:
            raise ValueError("Invalid method. Choose 'IQR', 'Z-score', or 'StdDev'.")
        
        outlier_bounds[column] = {'lower_bound': lower_bound, 'upper_bound': upper_bound}
    
    return outlier_bounds

# קריאה לפונקציה והדפסת התוצאות
outlier_bounds = calculate_outlier_bounds(df_chek_outliers)
for column, bounds in outlier_bounds.items():
    print(f"Column: {column}")
    print(f"Lower Bound: {bounds['lower_bound']}, Upper Bound: {bounds['upper_bound']}")


In [ ]:
df_after_chek_outliers= OutliersRemoveOrReplace(df_chek_outliers, method_outliers, threshold, Outliersaction)
df_after_chek_outliers

## פונקציה מספר 4 - השלמת חסרים

In [9]:
def Fill_missing_values(df, method_missing_values):
        
    # Count initial missing values
    initial_missing_count = df.isnull().sum().sum()
    
    if method_missing_values == 'mean':
        df_filled = df.fillna(df.mean())
    elif method_missing_values == 'median':
        df_filled = df.fillna(df.median())
    elif method_missing_values == 'drop_rows':
        df_filled = df.dropna()
    else:
        raise ValueError("Invalid method. Choose 'mean', 'median', or 'drop_rows'.")

    # Count remaining missing values
    remaining_missing_count = df_filled.isnull().sum().sum()

    # Calculate number of missing values handled
    num_filled = initial_missing_count - remaining_missing_count

    print("missing values method:", method_missing_values)
    print(f"Number of missing values handled: {num_filled}")
    print(f"Number of remaining missing values: {remaining_missing_count}")
    PercentageOfHandel = num_filled / df.size * 100
    if method_missing_values == 'drop_rows':
        print(f"Number of remaining rows: {df_filled.shape[0]}")
    print(f"Percentage of values handled: {PercentageOfHandel:.2f}%\n")

 

    return df_filled, num_filled, PercentageOfHandel

### בדיקה פנימית

In [ ]:
method_missing_values='mean'
df_after_Fill_missing_values, num_filled, PercentageOfHandel = Fill_missing_values(df_after_Clean_outliers, method_missing_values)
df_after_Fill_missing_values.head(5)


### בדיקה של השלמת חוסרים

In [ ]:
# Setting the seed for reproducibility
np.random.seed(50)

# Creating a DataFrame with 5 columns and 10 rows with some missing values
data_chek_missing = {
    'Apple': np.random.choice([np.nan, *range(10, 110, 10)], 20),
    'Banana': np.random.choice([np.nan, *range(5, 55, 5)], 20),
    'Cherry': np.random.choice([np.nan, *range(10, 110, 10)], 20),
    'Pear': np.random.choice([np.nan, *range(5, 105, 10)], 20),
    'Elderberry': np.random.choice([np.nan, *range(12, 120, 12)], 20)
}


df_chek_missing = pd.DataFrame(data_chek_missing)
df_chek_missing

In [ ]:
mean_values = df_chek_missing.mean()
print("\nMean values of each product:")
print(mean_values)

df_chek_missing.isnull().sum()

In [ ]:
df_after_chek_missing, w, z = Fill_missing_values(df_chek_missing, method_missing_values)
print(df_after_chek_missing)

## פונקציה מספר 5 - ולידאציה של היפר פרמטים

In [11]:
def safe_map(func, value_str):
    if value_str.strip('[]'):
        return list(map(func, value_str.strip('[]').split(',')))
    else:
        return []


## פונקציה מספר 6 - סינון שורות לפי שנים

In [13]:
def FilterByYears(df, years):
    # בדיקת קיום עמודת השנה
    if 'YEAR' not in df.columns:
        print("Error: 'YEAR' column does not exist in DataFrame")
        return None
    
    # המרת years לרשימה אם זה לא רשימה כבר
    if not isinstance(years, list):
        years = [years]
    
    df['YEAR'] = df['YEAR'].astype(str).str.strip()
    df = df[df['YEAR'].str.isnumeric()]
    
    available_years = set(df['YEAR'].unique())
    missing_years = [year for year in years if year not in available_years]
    
    if missing_years:
        print(f"Warning: The following years are not available in the DataFrame: {missing_years}")
    
    # סינון ה-DataFrame לפי השנים שהתקבלו
    filtered_df = df[df['YEAR'].isin(years)].copy()
    filtered_df['YEAR'] = filtered_df['YEAR'].astype(int)
    print(f"Years selected for filtering: {years}")
    return filtered_df

### בדיקה פנימית

In [ ]:
years_to_filter = ['2014','2015','2016']
type(years_to_filter)
print(years_to_filter)

In [ ]:
after_FilterByYears_df = FilterByYears(df_after_Fill_missing_values, years_to_filter)


In [ ]:
unique_years = after_FilterByYears_df['YEAR'].unique()
print(unique_years)
after_FilterByYears_df.head(2)

## פונקציה מספר 7  - נרמול ערכים

In [15]:
def Normalize_data(df, normalize_method):
    
    if normalize_method == 'Min-Max':
        scaler = MinMaxScaler()
        print("Normalization method: Min-Max\n")
    elif normalize_method == 'Quartile':
        print("Normalization method: Quartile\n")
        scaler = None
    else:
        print("Invalid normalization method. Choose 'Min-Max' or 'Quartile'.")
        return df
    
    # הפרדת העמודה האחרונה
    last_column = df.iloc[:, -1]
    columns_to_normalize = df.iloc[:, :-1]
    
    if normalize_method == 'Min-Max':
        # נרמול העמודות מלבד העמודה האחרונה
        normalized_data = pd.DataFrame(scaler.fit_transform(columns_to_normalize), columns=columns_to_normalize.columns)
    elif normalize_method == 'Quartile':
        # חישוב רבעונים
        Q1 = columns_to_normalize.quantile(0.25)
        Q3 = columns_to_normalize.quantile(0.75)
        IQR = Q3 - Q1
        normalized_data = (columns_to_normalize - Q1) / IQR
    
    # הוספת העמודה האחרונה חזרה ל-DataFrame
    normalized_data = pd.concat([normalized_data, last_column.reset_index(drop=True)], axis=1)
       
    return normalized_data

### בדיקה פנימית


In [ ]:
#בדיקה פנימית
normalize_method  = 'Min-Max'
normalize_method  = 'Quartile'
df_after_Normalize_data = Normalize_data(after_FilterByYears_df, normalize_method)
df_after_Normalize_data.head(3)

## XY פונקציה מספר 8 - פיצול סט מבחן וחלוקה ל

In [17]:
def SplitDataAndPrepForModel(df, test_size=0.2 ):
    
    columns_to_drop = ['YEAR']
    
    df_after_drop = df.drop(columns=columns_to_drop)
    # Shuffle the DataFrame
    df_shuffled = shuffle(df_after_drop, random_state=42)

    X = df_shuffled.iloc[:, :-1] # הסרת העמודה האחרונה
    y = df_shuffled.iloc[:, -1]  # בחירת העמודה האחרונה  

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)


    return X_train, X_test, y_train, y_test

בדיקה פנימית

In [ ]:
X_train, X_test, y_train, y_test = SplitDataAndPrepForModel(df_after_Normalize_data, test_size=0.2)


## פונקציה 9 - אימון וולדיצה של המודל

In [19]:
def train_and_evaluate_model(X_train, X_test, y_train, y_test, param_grid, model_name, expriment_number, cv=3):
    if model_name == 'random_forest':
        model = RandomForestRegressor()
    elif model_name == 'linear_regression':
        model = LinearRegression()
    elif model_name == 'xgboost':
        model = XGBRegressor()
    else:
        raise ValueError(f"Unsupported model: {model_name}")
    
    print("model_name:", model_name)
    print("cv:", cv)

    grid_search = GridSearchCV(model, param_grid, cv=cv, scoring='neg_mean_squared_error')
    grid_search.fit(X_train, y_train)

    best_model = grid_search.best_estimator_

    # Get cross-validation scores
    scores = cross_val_score(best_model, X_train, y_train, cv=cv, scoring='neg_mean_squared_error')
    mean_score = scores.mean()
    std_score = scores.std()  # Standard deviation of the cross-validation scores
    rmse_score = np.sqrt(-scores)  # RMSE for each fold in cross-validation

    # Evaluate on test data
    r_squared = best_model.score(X_test, y_test)  # R^2 on test data
    y_pred = best_model.predict(X_test)
    mse_test = mean_squared_error(y_test, y_pred)
    mae_test = mean_absolute_error(y_test, y_pred)
    rmse_test = np.sqrt(mse_test)  # RMSE on test data
    std_test = np.std(y_pred)  # Standard deviation of the test predictions
    
    print("best_model:", best_model)
    print("best_params:", best_model.get_params())
    print("cross_val_scores:", scores)
    print("mean_cross_val_score:", mean_score)
    print("std_cross_val_score:", std_score)
    print("mean_cross_val_rmse:", rmse_score.mean())
    print("r2_score:", r_squared)
    print("mean_squared_error_test:", mse_test)
    print("mean_absolute_error_test:", mae_test)
    print("root_mean_squared_error_test:", rmse_test)
    print("std_test:", std_test)
    
    results = {
        'best_model': best_model,
        'cross_val_scores': scores,
        'mean_cross_val_score': mean_score,
        'std_cross_val_score': std_score,
        'mean_cross_val_rmse': rmse_score.mean(),
        'r_squared': r_squared,
        'best_params': best_model.get_params(),
        'mse_test': mse_test,
        'mae_test': mae_test,
        'rmse_test': rmse_test,
        'std_test': std_test
    }
    
    # Save the best model using pickle
    output_path = './outputData/'
    model_filename = f'Output_ExperimentNumber{expriment_number}.pkl'
    model_filepath = os.path.join(output_path, model_filename)
    with open(model_filepath, 'wb') as f:
        pickle.dump(best_model, f)
    
    return results

## פונקציה - יוצרת גריד פרמטרים

In [21]:
def get_param_grid(model_name, model_params):
    param_grid = {}
    
    if model_name == 'random_forest':
        param_grid = {key: model_params[key] for key in model_params.keys() & {'n_estimators', 'max_depth', 'min_samples_split', 'min_samples_leaf'}}
    elif model_name == 'linear_regression':
        param_grid = {}
    elif model_name == 'xgboost':
        param_grid = {key: model_params[key] for key in model_params.keys() & {'n_estimators', 'max_depth', 'learning_rate', 'colsample_bytree', 'reg_lambda'}}
    else:
        raise ValueError(f"Unsupported model: {model_name}")
        
    print('The param grid for '+model_name+ ' model is:\n' )
    print(param_grid)


    return param_grid


## פונקציה 10 -  יוצרת פאנל, מפעילה את כל שאר הפונקציות

In [23]:
# מהופכת את הפאנל ,  להיות מוכן למודל

def Make_final_panel (panel_path,disease ,feature_types,method_outliers,threshold,Outliersaction,method_missing_values,normalize_method,years):
    
    OG_panel_df = pd.read_csv(panel_path)       
    after_FilterByYears_df = FilterByYears(OG_panel_df,years)
    after_selectFeatures_df = SelectFeatures(after_FilterByYears_df, feature_types, disease)
    after_CleanOutliers_df = OutliersRemoveOrReplace( after_selectFeatures_df , method_outliers, threshold,Outliersaction)
    after_missingValues_df,num_filled, remaining_missing_count = Fill_missing_values(after_CleanOutliers_df, method_missing_values)
    after_NormalizeData_df = Normalize_data(after_missingValues_df, normalize_method)
    
    return after_NormalizeData_df

    

 



# פונקציה סופית

In [25]:

# הפונקציה הראשית שמבצעת את כל הניסויים
def Final_fun(file_path_of_params, panel_path):
    
    # קריאת קובץ האקסל המכיל את פרמטרי הניסויים
    ExprimentTracker_df = pd.read_excel(file_path_of_params, keep_default_na=False,skiprows = 8 )
    
   
    # יצירת DataFrame ריק עם כותרות מתאימות
    all_results = pd.DataFrame(columns=['ExperimentNumber'] + list(ExprimentTracker_df.columns))



    # קריאת תוצאות קיימות אם הקובץ קיים
 #   if os.path.exists(results_filename):
  #      all_results = pd.read_excel(results_filename)
    
    # מעבר על כל שורה בקובץ האקסל של פרמטרי הניסויים
    for index, row in ExprimentTracker_df.iterrows():
        
        
        # שליפת פרטי הניסוי הנוכחי
        experiment_number = int(row['ExperimentNumber'])  # מספר הניסוי
        years = row['Years'].split(', ')  # השנים הנכללות בניסוי
        target = row['target']  # מטרה
        feature_types = row['feature_types'].split(', ')  # סוגי מאפיינים
        method_outliers = row['method_outliers']  # שיטת טיפול בנקודות חריגות
        Outliersaction = row['Outliersaction'] # מחיקה של שורה או לשים ערך ריק
        threshold = row['threshold']  # סף
        method_missing_values = row['method_missing_values']  # שיטת טיפול בערכים חסרים
        normalize_method = row['normalize_method']  # שיטת נורמליזציה
        model_name = row['model_name']  # שם המודל
        n_estimators = safe_map(int, str(row['n_estimators']))  # מספר האסימאטורים
        max_depth = safe_map(int, str(row['max_depth']))  # עומק מקסימלי
        learning_rate = safe_map(float, str(row['learning_rate']))  # שיעור למידה
        min_samples_split = safe_map(int, str(row['min_samples_split']))  # מינימום דוגמאות לפיצול
        min_samples_leaf = safe_map(int, str(row['min_samples_leaf']))  # מינימום דוגמאות בעלה
        colsample_bytree = safe_map(float, str(row['colsample_bytree']))  # דגימה לפי עץ
        reg_lambda = safe_map(float, str(row['reg_lambda']))  # פרמטר רגולריזציה
        cv = int(row['cv'])  # קרוס ולידציה
        
        # הדפסת מידע על הניסוי הנוכחי
        print('\n', 25*"#", " Experiment Number ", experiment_number, 25*"#")

        # יצירת הפאנל הסופי על פי הפרמטרים
        final_panel = Make_final_panel(panel_path, target, feature_types, method_outliers, threshold,Outliersaction, method_missing_values, normalize_method, years)

        # חלוקת הנתונים לערכות אימון ובדיקה
        X_train, X_test, y_train, y_test = SplitDataAndPrepForModel(final_panel, test_size=0.2)
        
        # הגדרת פרמטרי המודל
        model_params = {
            'n_estimators': n_estimators,
            'max_depth': max_depth,
            'learning_rate': learning_rate,
            'min_samples_split': min_samples_split,
            'min_samples_leaf': min_samples_leaf,
            'colsample_bytree': colsample_bytree,
            'reg_lambda': reg_lambda,
        }
        
        # יצירת גריד של פרמטרים למודל
        param_grid = get_param_grid(model_name, model_params)
        
        start_time = time.time()
        
        # אימון והערכת המודל
        results = train_and_evaluate_model(X_train, X_test, y_train, y_test, param_grid, model_name,  experiment_number,cv)
        
        end_time = time.time()
        total_time_seconds = end_time - start_time
        total_time_minutes = total_time_seconds / 60  # Convert seconds to minutes
        
        # וידוא שהתוצאות הן מילון
        if not isinstance(results, dict):
            raise ValueError("The results from train_and_evaluate_model should be a dictionary")

        # יצירת שורת נתונים חדשה עם תוצאות הניסוי
        new_row = row.to_dict()
        new_row.update(results)
        new_row['date_time'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        new_row['RunTimeMinutes'] = total_time_minutes        
        new_row['num_columns'] = (final_panel.shape[1]-1)
        new_row['num_rows'] = final_panel.shape[0]
      
        new_row_df = pd.DataFrame([new_row])
        print("Running time in minutes  : ",total_time_minutes  )
        
        # וידוא שהעמודות ייחודיות
        all_results = all_results.loc[:,~all_results.columns.duplicated()]
        new_row_df = new_row_df.loc[:,~new_row_df.columns.duplicated()]

        # בדיקת עמודות חסרות והוספת ערכי None
        missing_cols = set(all_results.columns) - set(new_row_df.columns)
        for col in missing_cols:
            new_row_df[col] = None
        
        # הוספת השורה החדשה ל-DataFrame של כל התוצאות
        all_results = pd.concat([all_results, new_row_df], ignore_index=True)
        
            # שם הקובץ לשמירת התוצאות
        results_filename = "Output_ExprimentResults_"+model_name+".xlsx"
        
        # שמירת התוצאות לקובץ אקסל
        all_results.to_excel(results_filename, index=False)
        #all_results.info()

              
    return all_results


# הרצה של פונקציה סופית

In [35]:
panel_path = 'InputData\Panel_06.csv'
ExprimentTracker_path ='InputData\‏‏ExprimentTracker_Xgboost_with2013.xlsx'
test = Final_fun (ExprimentTracker_path,panel_path)



 #########################  Experiment Number  1 #########################
Years selected for filtering: ['2013', '2014', '2015', '2016', '2017', '2018', '2019']
Selected Categories :

Gender:
Total_All, MALE_All, FEMALE_All, Male_0_4, Male_5_9, Male_10_14, Male_15_19, Male_20_24, Male_25_29, Male_30_34, Male_35_39, Male_40_44, Male_45_49, Male_50_54, Male_55_59, Male_60_64, Male_65_69, Male_70_74, Male_75_79, Male_80_84, Male_85+, Female_0_4, Female_5_9, Female_10_14, Female_15_19, Female_20_24, Female_25_29, Female_30_34, Female_35_39, Female_40_44, Female_45_49, Female_50_54, Female_55_59, Female_60_64, Female_65_69, Female_70_74, Female_75_79, Female_80_84, Female_85+
Description: Demographic data based on gender

Income:
Weighted_Total_annual_income_(£), Weighted_Net_annual_income_(£), Weighted_Net_annual_income_before_housing_costs_(£), Weighted_Net_annual_income_after_housing_costs_(£)
Description: Economic data related to income

Deprivation:
Weighted_Index_of_Multiple_Depriva

In [36]:
print("hoo")

hoo
